In [ ]:
import argparse
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow import keras


class WindowGenerator:
    def __init__(self, input_width, label_options, mean, std):
        self.input_width = input_width  # the number of samples contained in a single window
        self.label_options = label_options
        self.mean = tf.reshape(tf.convert_to_tensor(mean), [1, 1, 2])  # sec arg is the shape and during training the first dim is batch
        self.std = tf.reshape(tf.convert_to_tensor(std), [1, 1, 2])

    def split_window(self, features):
        # here the assumption is that features already contains 7 values
        inputs = features[:, :-1, :]  # we leave the last one as label, remember we have the batch at the first dim

        if self.label_options < 2:
            labels = features[:, -1, self.label_options]
            labels = tf.expand_dims(labels, -1)
            num_labels = 1
        else:
            labels = features[:, -1, :]
            num_labels = 2

        inputs.set_shape([None, self.input_width, 2])  # we set the batch dim as None because it will be needed later
        labels.set_shape([None, num_labels])  # same for labels

        return inputs, labels

    def normalize(self, features):
        features = (features - self.mean) / (self.std + 1.e-6)  # we add a small quantity to avoid divisions by zero

        return features

    def preprocess(self, features):
        inputs, labels = self.split_window(features)
        inputs = self.normalize(inputs)  # we don't need to normalize labels

        return inputs, labels

    def make_dataset(self, data, train):
        ds = tf.keras.preprocessing.timeseries_dataset_from_array( # this is what we need
                data=data,
                targets=None,
                sequence_length=self.input_width+1,  # + label
                sequence_stride=1,
                batch_size=32)  # we'll try to change batch size later
        ds = ds.map(self.preprocess)  # the preprocess will be applied to every batch
        ds = ds.cache()

        if train is True:
            ds = ds.shuffle(100, reshuffle_each_iteration=True) # 100 is the shuffling buffer size

        return ds


def train_model(model, labels, train_ds, val_ds, test_ds, epochs=10, saved_model_dir=None, alpha=1):

    final_units = max(1, labels)
    print(alpha)
    if model == 'mlp':
        model = keras.Sequential([
            keras.layers.Flatten(),
            keras.layers.Dense(units=int(alpha*128)),
            keras.layers.ReLU(),
            keras.layers.Dense(units=int(alpha*128)),
            keras.layers.ReLU(),
            keras.layers.Dense(units=final_units)
        ])
    elif model == 'cnn':
        model = keras.Sequential([
            keras.layers.Conv1D(filters=int(alpha*64), kernel_size=3),
            keras.layers.ReLU(),
            keras.layers.Flatten(),
            keras.layers.Dense(units=int(alpha*64)),
            keras.layers.ReLU(),
            keras.layers.Dense(units=final_units)
        ])
    elif model == 'lstm':
        model = keras.Sequential([
            keras.layers.LSTM(units=int(alpha*64)),
            keras.layers.Flatten(),
            keras.layers.Dense(units=final_units)
        ])

    if labels >= 2:
        new_mae = customMAE()
        metrics = [new_mae]
    else:
        metrics = [keras.metrics.MeanAbsoluteError()]

    model.compile(
        optimizer='adam',
        loss=keras.losses.MeanSquaredError(),
        metrics=metrics
    )

    # let's train for 20 epochs

    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)
    test_loss, test_mae = model.evaluate(test_ds)
    print("{} \n The MAE is: {}".format(model.summary(), test_mae))

    if saved_model_dir is not None:
        run_model = tf.function(lambda x: model(x))
        concrete_func = run_model.get_concrete_function(tf.TensorSpec([1, 6, 2], tf.float32))
        model.save(args.saved_model_dir, signatures=concrete_func)


class customMAE(keras.metrics.Metric):
    def __init__(self, name='custom_MAE', **kwargs):
        super().__init__(name=name, **kwargs)
        # MAE: we need the sum --> total, and the count --> count to compute the mean
        self.total = self.add_weight('total', initializer='zero', shape=(2,))
        self.count = self.add_weight('count', initializer='zeros')

    # We have to iterate over all the dataset and update the state vars
    # This is computed at every batch
    def update_state(self, y_true, y_pred, sample_weight=None):
        error = tf.abs(y_pred-y_true)
        error = tf.reduce_mean(error, axis=0)
        self.total.assign_add(error)
        self.count.assign_add(1.)
        return

    def reset_states(self):
        self.count.assign(tf.zeros_like(self.count))
        self.total.assign(tf.zeros_like(self.total))
        return

    # after we have updated for all the dataset we return the result
    def result(self):
        result = tf.math.divide_no_nan(self.total, self.count)
        return result

In [ ]:
    parser = argparse.ArgumentParser()
    parser.add_argument('--model', type=str, help='model name among mlp cnn and lstm')
    parser.add_argument('--labels', type=int, default=0,
                        help='0 for temp forecasting, 1 for hum forecasting, 2 or more for both')
    parser.add_argument('--saved_model_dir', type=str, default=None)
    parser.add_argument('--alpha', type=float, default=1, help='Width multiplaier for structured pruning via width scaling')
    args = parser.parse_args(['--model=cnn', '--labels=2', '--saved_model_dir=cnn_struct_pruning_5', '--alpha=0.5'])

    seed = 42
    tf.random.set_seed(seed)
    np.random.seed(seed)

    zip_path = tf.keras.utils.get_file(
        origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
        fname='jena_climate_2009_2016.csv.zip',
        extract=True,
        cache_dir='.', cache_subdir='data')

    csv_path, _ = os.path.splitext(zip_path)
    df = pd.read_csv(csv_path)

    column_indices = [2, 5]
    columns = df.columns[column_indices]
    data = df[columns].values.astype(np.float32)

    n = len(data)
    train_data = data[0:int(n * 0.7)]
    val_data = data[int(n * 0.7):int(n * 0.9)]
    test_data = data[int(n * 0.9):]

    mean = train_data.mean(axis=0)
    std = train_data.std(axis=0)

    input_width = 6
    LABEL_OPTIONS = args.labels

    generator = WindowGenerator(input_width, LABEL_OPTIONS, mean, std)
    train_ds = generator.make_dataset(train_data, True)
    val_ds = generator.make_dataset(val_data, False)
    test_ds = generator.make_dataset(test_data, False)

    train_model(args.model, LABEL_OPTIONS, train_ds, val_ds, test_ds, 20,
                args.saved_model_dir, args.alpha)


0.5
Epoch 1/20
9195/9200 [============================>.] - ETA: 0s - loss: 62.0796 - custom_MAE: 2.3945

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1841: UserWarning: Metric customMAE implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


9200/9200 [==============================] - 83s 9ms/step - loss: 62.0468 - custom_MAE: 2.3936 - val_loss: 1.9633 - val_custom_MAE: 0.9774
Epoch 2/20
9200/9200 [==============================] - 21s 2ms/step - loss: 0.9514 - custom_MAE: 0.6004 - val_loss: 0.7542 - val_custom_MAE: 0.5502
Epoch 3/20
9200/9200 [==============================] - 25s 3ms/step - loss: 0.8403 - custom_MAE: 0.5461 - val_loss: 0.5847 - val_custom_MAE: 0.4302
Epoch 4/20
9200/9200 [==============================] - 24s 3ms/step - loss: 0.7210 - custom_MAE: 0.5066 - val_loss: 0.5877 - val_custom_MAE: 0.4320
Epoch 5/20
9200/9200 [==============================] - 25s 3ms/step - loss: 0.7004 - custom_MAE: 0.4941 - val_loss: 0.6705 - val_custom_MAE: 0.4748
Epoch 6/20
9200/9200 [==============================] - 24s 3ms/step - loss: 0.6879 - custom_MAE: 0.4803 - val_loss: 1.6774 - val_custom_MAE: 0.9351
Epoch 7/20
9200/9200 [==============================] - 25s 3ms/step - loss: 0.6767 - custom_MAE: 0.4734 - val_loss:

In [ ]:
    parser = argparse.ArgumentParser()
    parser.add_argument('--model', type=str, help='model name among mlp cnn and lstm')
    parser.add_argument('--labels', type=int, default=0,
                        help='0 for temp forecasting, 1 for hum forecasting, 2 or more for both')
    parser.add_argument('--saved_model_dir', type=str, default=None)
    parser.add_argument('--alpha', type=float, default=1, help='Width multiplaier for structured pruning via width scaling')
    args = parser.parse_args(['--model=cnn', '--labels=2', '--saved_model_dir=cnn_struct_pruning_75', '--alpha=0.75'])

    seed = 42
    tf.random.set_seed(seed)
    np.random.seed(seed)

    zip_path = tf.keras.utils.get_file(
        origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
        fname='jena_climate_2009_2016.csv.zip',
        extract=True,
        cache_dir='.', cache_subdir='data')

    csv_path, _ = os.path.splitext(zip_path)
    df = pd.read_csv(csv_path)

    column_indices = [2, 5]
    columns = df.columns[column_indices]
    data = df[columns].values.astype(np.float32)

    n = len(data)
    train_data = data[0:int(n * 0.7)]
    val_data = data[int(n * 0.7):int(n * 0.9)]
    test_data = data[int(n * 0.9):]

    mean = train_data.mean(axis=0)
    std = train_data.std(axis=0)

    input_width = 6
    LABEL_OPTIONS = args.labels

    generator = WindowGenerator(input_width, LABEL_OPTIONS, mean, std)
    train_ds = generator.make_dataset(train_data, True)
    val_ds = generator.make_dataset(val_data, False)
    test_ds = generator.make_dataset(test_data, False)

    train_model(args.model, LABEL_OPTIONS, train_ds, val_ds, test_ds, 20,
                args.saved_model_dir, args.alpha)


0.75
Epoch 1/20
9188/9200 [============================>.] - ETA: 0s - loss: 47.5337 - custom_MAE: 2.0268

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1841: UserWarning: Metric customMAE implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


9200/9200 [==============================] - 81s 9ms/step - loss: 47.4744 - custom_MAE: 2.0254 - val_loss: 1.8380 - val_custom_MAE: 1.0547
Epoch 2/20
9200/9200 [==============================] - 29s 3ms/step - loss: 0.9262 - custom_MAE: 0.5893 - val_loss: 0.6302 - val_custom_MAE: 0.4825
Epoch 3/20
9200/9200 [==============================] - 27s 3ms/step - loss: 0.8529 - custom_MAE: 0.5405 - val_loss: 0.6314 - val_custom_MAE: 0.4577
Epoch 4/20
9200/9200 [==============================] - 27s 3ms/step - loss: 0.7260 - custom_MAE: 0.4998 - val_loss: 0.5694 - val_custom_MAE: 0.4283
Epoch 5/20
9200/9200 [==============================] - 29s 3ms/step - loss: 0.7107 - custom_MAE: 0.4926 - val_loss: 0.6931 - val_custom_MAE: 0.5133
Epoch 6/20
9200/9200 [==============================] - 26s 3ms/step - loss: 0.6938 - custom_MAE: 0.4807 - val_loss: 2.1906 - val_custom_MAE: 1.2628
Epoch 7/20
9200/9200 [==============================] - 26s 3ms/step - loss: 0.7005 - custom_MAE: 0.4775 - val_loss:

In [ ]:
# TfLite model generation
converter = tf.lite.TFLiteConverter.from_saved_model("cnn_struct_pruning_75/")
tflite_model = converter.convert()
with open("cnn_prun_075.tflite", 'wb') as fp:
  fp.write(tflite_model)
